In [1]:
import builtins
builtins.uclresearch_topic = 'GIVENCHY' # 226984 entires
# builtins.uclresearch_topic = 'HAWKING' # 4828104 entries
# builtins.uclresearch_topic = 'NYC'
# builtins.uclresearch_topic = 'FLORIDA'
from configuration import config
import numpy as np
import pandas as pd
import pickle
import os
import os.path
import math
import sys
import csv
import matplotlib.pyplot as plt
from sklearn.externals import joblib
import scipy.integrate as spi
import pylab as pl
import datetime as dt
from datetime import timezone
from datetime import timedelta
import networkx as nx
from tqdm import tqdm
import pandas as pd
import numpy as np
import math
import multiprocessing
from multiprocessing import Pool
from IPython.display import display


('Reading config file from location: '
 '/Users/jay/MSC_WSBDA/MSc_Thesis/Msc_project/configuration/env.properties')
{'calculate': {'analysis': True,
               'network': True,
               'uniquetweets': True,
               'uniqueusers': True},
 'data': {'dates': ['2018-03-11', '2018-03-12', '2018-03-13'],
          'eventname': "Givenchy's Death",
          'phrases': ['givenchy%20death', 'givenchy%20passed%20away'],
          'starttime': 'Mar 12 08:20:00 -0500 2018'},
 'path': {'crawl': {'followers': '/Users/jay/MSC_WSBDA/MSc_Thesis/Msc_project/followers',
                    'friends': '/Users/jay/MSC_WSBDA/MSc_Thesis/Msc_project/friends'},
          'cwd': '/Users/jay/MSC_WSBDA/MSc_Thesis/Msc_project/data/givenchy',
          'ml': '/Users/jay/MSC_WSBDA/MSc_Thesis/Msc_project/data/givenchy/pickle',
          'networkx': {'all': '/Users/jay/MSC_WSBDA/MSc_Thesis/Msc_project/data/givenchy/pickle/networkx_all.dat',
                       'friends': '/Users/jay/MSC_WSBDA/MSc_

In [2]:
n_bytes = 2**31
max_bytes = 2**31 - 1

path = "/Users/jay/MSC_WSBDA/MSc_Thesis/Msc_project/Data/"

def write_to_pickle(file_path, data):

    bytes_out = pickle.dumps(data)
    with open(file_path, 'wb') as f_out:
        for idx in range(0, n_bytes, max_bytes):
            f_out.write(bytes_out[idx:idx+max_bytes])

def load_pickle(file_path):
    bytes_in = bytearray(0)
    input_size = os.path.getsize(file_path)
    with open(file_path, 'rb') as f_in:
        for _ in range(0, input_size, max_bytes):
            bytes_in += f_in.read(max_bytes)
    
    return pickle.loads(bytes_in)

In [281]:
unique_users = load_pickle(path+"80000users_2.pkl")
unique_users.head()

,contributors_enabled,created_at,description,favourites_count,followers_count,followers_json,friends_count,friends_json,geo_enabled,id,...,listed_count,location,name,protected,screen_name,statuses_count,url,verified,source_candidates,target_nodes
0,0,2010-11-11 03:59:09,,548,2090,None,46,None,1,214328887,...,65,,солнечная,1,xsolnechnaya,201,None,0,"[105, 62, 147, 148, 42, 124, 32, 10, 96, 18, 1...","[34428380, 56860418, 157829215, 158419434, 149..."
1,0,2009-04-22 22:44:05,https://t.co/XA34weUL7o Join Now for Exclusive...,38,305257,None,181,None,1,34428380,...,4929,"Nashville, TN",JERMbeats,0,schzimmydeanie,5695,https://t.co/XA34weUL7o,1,"[0, 12, 11, 110, 33, 118, 167, 78, 18, 74, 90,...","[31331740, 21548772, 17759158, 17868918, 40981..."
2,0,2008-11-02 17:07:55,Paramore Spain. primera web en español sobre P...,31,11756,None,498,None,0,17116707,...,93,España,ParamoreSpain,0,paramorespain,5124,https://t.co/s40bqOYO0F,0,"[18, 56, 52, 111, 153, 97, 38, 35, 31, 169, 20...","[28465635, 231238695, 121533789, 86221475, 808..."
3,0,2009-04-03 01:03:15,A maior fonte de informações sobre a banda Par...,4449,57992,None,1482,"[823625606705410052, 62089415, 976610561126490...",1,28465635,...,894,Brasil,Paramore Brasil,0,paramorebrasil,35499,https://t.co/0prUO4vpzM,0,"[2, 111, 36, 158, 83, 118, 179, 148, 65, 130, ...","[30971165, 297801196, 259842341, 123371682, 19..."
4,0,2011-09-26 21:58:16,Your Swedish/worldwide source of everything Pa...,1968,15033,None,219,None,1,380580781,...,45,Sweden / Worldwide,🙃 Paramore Sweden 🇸🇪,0,SWEparamore,20720,None,0,"[18, 96, 35, 0, 66, 32, 156, 75, 55, 54, 25, 4...","[18996905, 400689940, 31331740, 19358562, 2043..."


In [282]:
unique_users=unique_users.drop(['contributors_enabled','description','location','screen_name','name','url','verified',
                                'protected','lang','geo_enabled','friends_json','followers_json'
                               ],axis=1)

In [283]:
a=unique_users[unique_users.created_at.isin(['0000-00-00 00:00:00'])].index.values
unique_users.loc[a,'created_at']='2010-11-11 03:59:09'

In [284]:
start_timestamp = dt.datetime.strptime(config.settings['data']['starttime'], '%b %d %H:%M:%S %z %Y')

def convert_utc_to_est(time_string):
    datetime_object = dt.datetime.strptime(time_string, '%Y-%m-%d %H:%M:%S')
    return datetime_object.replace(tzinfo=timezone.utc).astimezone(tz=timezone(-timedelta(hours=5)))


def get_user_created_days(created_at):
    delta_time = start_timestamp - convert_utc_to_est(str(created_at))
    return max(delta_time.days, 0)+1



In [285]:
unique_users = unique_users.rename(columns={'id': 'user_id'})
unique_users = unique_users.rename(columns={'statuses_count': 'user_statuses_count'})
unique_users = unique_users.rename(columns={'favourites_count': 'user_favourites_count'})
unique_users = unique_users.rename(columns={'listed_count': 'user_listed_count'})



In [286]:
unique_users['user_created_days'] = list(map(lambda created_at: get_user_created_days(created_at), unique_users.created_at))
unique_users['normalized_user_statuses_count'] = np.divide(unique_users.user_statuses_count, unique_users.user_created_days)
unique_users['normalized_user_followers_count'] = np.divide(unique_users.followers_count, unique_users.user_created_days)
unique_users['normalized_user_favourites_count'] = np.divide(unique_users.user_favourites_count, unique_users.user_created_days)
unique_users['normalized_user_listed_count'] = np.divide(unique_users.user_listed_count, unique_users.user_created_days)
unique_users['normalized_user_friends_count'] = np.divide(unique_users.friends_count, unique_users.user_created_days)



In [291]:
unique_users.to_csv(path+'70013users.csv',index=False)

In [288]:
unique_users.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 70013 entries, 0 to 70012
Data columns (total 15 columns):
created_at                          70013 non-null object
user_favourites_count               70013 non-null int64
followers_count                     70013 non-null int64
friends_count                       70013 non-null int64
user_id                             70013 non-null int64
user_listed_count                   70013 non-null int64
user_statuses_count                 70013 non-null int64
source_candidates                   70013 non-null object
target_nodes                        70013 non-null object
user_created_days                   70013 non-null int64
normalized_user_statuses_count      70013 non-null float64
normalized_user_followers_count     70013 non-null float64
normalized_user_favourites_count    70013 non-null float64
normalized_user_listed_count        70013 non-null float64
normalized_user_friends_count       70013 non-null float64
dtypes: float64(5), int

In [292]:
df = pd.read_csv(path+'70013users.csv')

In [293]:
df

,created_at,user_favourites_count,followers_count,friends_count,user_id,user_listed_count,user_statuses_count,source_candidates,target_nodes,user_created_days,normalized_user_statuses_count,normalized_user_followers_count,normalized_user_favourites_count,normalized_user_listed_count,normalized_user_friends_count
0,2010-11-11 03:59:09,548,2090,46,214328887,65,201,"[105, 62, 147, 148, 42, 124, 32, 10, 96, 18, 1...","[34428380, 56860418, 157829215, 158419434, 149...",2679,0.075028,0.780142,0.204554,0.024263,0.017171
1,2009-04-22 22:44:05,38,305257,181,34428380,4929,5695,"[0, 12, 11, 110, 33, 118, 167, 78, 18, 74, 90,...","[31331740, 21548772, 17759158, 17868918, 40981...",3246,1.754467,94.040974,0.011707,1.518484,0.055761
2,2008-11-02 17:07:55,31,11756,498,17116707,93,5124,"[18, 56, 52, 111, 153, 97, 38, 35, 31, 169, 20...","[28465635, 231238695, 121533789, 86221475, 808...",3417,1.499561,3.440445,0.009072,0.027217,0.145742
3,2009-04-03 01:03:15,4449,57992,1482,28465635,894,35499,"[2, 111, 36, 158, 83, 118, 179, 148, 65, 130, ...","[30971165, 297801196, 259842341, 123371682, 19...",3266,10.869259,17.756277,1.362217,0.273729,0.453766
4,2011-09-26 21:58:16,1968,15033,219,380580781,45,20720,"[18, 96, 35, 0, 66, 32, 156, 75, 55, 54, 25, 4...","[18996905, 400689940, 31331740, 19358562, 2043...",2359,8.783383,6.372616,0.834252,0.019076,0.092836
5,2009-01-14 21:32:04,2158,284075,68,18996905,1934,14602,"[4, 64, 42, 173, 66, 155, 153, 112, 43, 156, 2...","[15023872, 18951737, 17868918, 34428380, 22462...",3344,4.366627,84.950658,0.645335,0.578349,0.020335
6,2010-11-29 15:23:21,309,2121,206,221036078,12,6933,"[148, 8, 83, 68, 46, 0, 124, 113, 116, 33, 31,...","[153460275, 22462180, 196327549, 117674417, 29...",2660,2.606391,0.797368,0.116165,0.004511,0.077444
7,2010-06-08 16:33:07,74,4710,35,153460275,49,807,"[6, 115, 31, 23, 82, 62, 44, 18, 20, 173, 14, ...","[43003845, 19493072, 22462180, 34428380, 15133...",2834,0.284757,1.661962,0.026112,0.017290,0.012350
8,2010-01-23 22:17:51,350,11627,934,107830991,63,3021,"[10, 61, 35, 28, 41, 56, 52, 3, 49, 113, 69, 9...","[17868918, 40981798, 100581193, 461410856, 274...",2970,1.017172,3.914815,0.117845,0.021212,0.314478
9,2008-12-04 14:49:42,9,16996,552,17868918,238,4153,"[8, 87, 123, 20, 37, 16, 66, 95, 80, 111, 69, ...","[31331740, 8088112, 117674417, 17759158, 40981...",3385,1.226883,5.020975,0.002659,0.070310,0.163072


In [197]:
write_to_pickle(path+'/80000users_3.pkl', unique_users)

In [198]:
sf = load_pickle('Stanford_features.pkl')

## Creating a graph from edgelist and then finding avg neighbour degree of nodes

In [43]:
G=nx.read_edgelist(path+"final_stanford_edgelist.csv",delimiter = ",",nodetype=int)

In [35]:
J=nx.read_edgelist(path+"final_stanford_edgelist.csv",create_using=nx.DiGraph(),delimiter = ",",nodetype=int)

In [44]:
G.number_of_nodes()

69734

In [45]:
G.number_of_edges()

1092287

In [46]:
edges = list(G.edges())

In [47]:
for i in edges:
    if i[0] == 36659758 or i[1] == 36659758:
        print(f"i:{i}")
        

i:(23828885, 36659758)
i:(34707065, 36659758)
i:(28147425, 36659758)
i:(55017958, 36659758)
i:(42495702, 36659758)
i:(41267619, 36659758)
i:(15515673, 36659758)
i:(34604820, 36659758)
i:(20337511, 36659758)
i:(43979384, 36659758)
i:(4003351, 36659758)
i:(44524453, 36659758)
i:(90919384, 36659758)
i:(27123742, 36659758)
i:(36659758, 73876706)
i:(36659758, 232383098)
i:(36659758, 15165411)
i:(36659758, 97218252)
i:(36659758, 18080534)
i:(36659758, 253823543)
i:(36659758, 21225900)


In [42]:
edgelist[(edgelist['Target'] == 36659758) | (edgelist['Source'] == 36659758)]

,Source,Target
513436,23828885,36659758
785450,73876706,36659758
785487,55017958,36659758
785504,232383098,36659758
785623,41267619,36659758
785658,44524453,36659758
785671,90919384,36659758
785709,34604820,36659758
785725,43979384,36659758
785762,4003351,36659758


In [24]:
edgelist = pd.read_csv(path+"final_stanford_edgelist.csv",names=['Source','Target'],header=None)

In [25]:
edgelist[edgelist['Source'] == 36659758]

,Source,Target
785817,36659758,28147425
785818,36659758,42495702
785819,36659758,34707065


In [13]:
edgelist1 = pd.read_csv(path+"final_stanford_edgelist.csv")

In [15]:
G = nx.DiGraph(data=edgelist1)

In [277]:
edgelist['Source'].unique()

62506

In [300]:
filtered_users = pd.read_csv(path+'filtered_70013users.csv')

In [301]:
seeds = filtered_users.iloc[0:10,:]

In [302]:
seeds.to_csv(path+'top_10_avg_neigh_degree_seeds.csv')

In [303]:
seeds

,created_at,user_favourites_count,followers_count,friends_count,user_id,user_listed_count,user_statuses_count,source_candidates,target_nodes,user_created_days,normalized_user_statuses_count,normalized_user_followers_count,normalized_user_favourites_count,normalized_user_listed_count,normalized_user_friends_count,avg_neighbour_degree
0,2009-04-26 23:49:25,0,9,22,35604056,0,0,[],[3359851],3242,0.000000,0.002776,0.000000,0.000000,0.006786,853.0
1,2010-11-17 20:55:07,0,7,20,216829577,0,65,[],"[59804598, 3359851]",2672,0.024326,0.002620,0.000000,0.000000,0.007485,843.0
2,2009-02-04 09:21:27,3669,5216,978,20039736,146,88227,[1238],[59804598],3324,26.542419,1.569194,1.103791,0.043923,0.294224,833.0
3,2011-10-07 08:28:59,20,641,165,386428938,12,456,"[24332, 525]","[59804598, 5442012]",2349,0.194125,0.272882,0.008514,0.005109,0.070243,752.0
4,2009-03-12 06:35:50,2316,185972,2567,23912766,688,6088,"[66249, 66250, 66243, 6327]",[813286],3288,1.851582,56.560827,0.704380,0.209246,0.780718,715.0
5,2009-10-08 15:31:01,3,102,54,80873238,0,259,"[3409, 3388, 9016]",[18581803],3077,0.084173,0.033149,0.000975,0.000000,0.017550,683.0
6,2009-03-14 23:01:51,0,74388,18684,24446042,4137,6655,"[43877, 1124, 43876, 1106, 1056, 43862, 43868,...",[5442012],3285,2.025875,22.644749,0.000000,1.259361,5.687671,671.0
7,2009-02-23 00:24:44,55,112,253,21614972,8,466,[],[15846407],3305,0.140998,0.033888,0.016641,0.002421,0.076551,595.0
8,2009-03-12 01:13:58,421,281,665,23867268,10,9669,[],"[42361118, 3359851]",3288,2.940693,0.085462,0.128041,0.003041,0.202251,514.5
9,2009-10-13 17:37:40,240,23143,34,82146778,589,530,"[13385, 61266, 61246, 3325, 61242, 1275, 13142...",[22679419],3072,0.172526,7.533529,0.078125,0.191732,0.011068,514.0
